# Quickstart

References
- https://python.langchain.com/docs/get_started/quickstart
- https://python.langchain.com/docs/integrations/llms/azure_openai

In [35]:
import dotenv
dotenv.load_dotenv()

True

## LLM Chain

In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [4]:
llm.invoke("how can langsmith help with testing?")

"\nLangsmith is a language model that can assist with testing in several ways:\n\n1. Automated Testing: Langsmith can be used to generate automated tests for software applications. It can analyze the application's code and generate test cases based on the syntax and semantics of the code. This can help save time and effort in writing manual tests, and also improve testing coverage and accuracy.\n2. Code Coverage Analysis: Langsmith can analyze the test cases generated by it or any other test suite, and provide information on the code coverage of the tests. It can identify areas of the code that are not being tested enough, or areas where tests are not effective in detecting errors.\n3. Test Data Generation: Langsmith can be used to generate test data that is relevant to a specific application or feature. For example, it can generate test data for a web application based on user interactions, or generate test cases for a mobile app based on real-world usage scenarios.\n4. Regression Tes

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm

In [7]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I must say that Langsmith is a versatile tool that can assist with testing in several ways. Here are some of the ways Langsmith can help:\n\n1. Automated Testing: Langsmith's AI-powered engine can automatically generate test cases based on your technical documentation. This can save you time and effort while ensuring comprehensive testing. By using Langsmith, you can automate repetitive tasks such as data entry, regression testing, and exploratory testing.\n2. Test Data Generation: Langsmith's natural language processing (NLP) capabilities allow it to generate test data that mimics real-world scenarios. This can help identify potential issues early on in the development cycle, ensuring that your product is thoroughly tested before release.\n3. Test Case Prioritization: Langsmith's AI-powered engine can prioritize test cases based on risk and complexity, ensuring that critical tests are executed first. This c

In [8]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser

In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that Langsmith is an incredible tool for testing purposes! Here are some ways in which Langsmith can help with testing:\n\n1. Automated Content Creation: With Langsmith, you can create automated content based on your existing documentation, such as API documentation, user manuals, and more. This can save time and effort during the testing process by reducing the amount of manual work involved in creating test cases.\n2. Smart Content Analysis: Langsmith's AI-powered content analysis tools can help you identify potential issues with your documentation before they become critical problems. By analyzing your content for consistency, completeness, and accuracy, Langsmith can help you catch mistakes that might otherwise go unnoticed during the testing process.\n3. Intelligent Testing: With Langsmith's intelligent testing capabilities, you can create more accurate and comprehensive test cases by analyzing your content and identif

## Retrieval Chain

In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, langsmith can help with testing by allowing users to visualize their test results. This means that langsmith can provide a visual representation of the test results, making it easier for users to understand and analyze the data. This can be particularly useful for complex tests or large amounts of data, as it allows users to quickly identify trends or patterns in the results.'

In [18]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrival_chain = create_retrieval_chain(retriever, document_chain)

In [19]:
response = retrival_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])


Based on the provided context, LangSmith can help with testing in several ways:

1. Prototyping: LangSmith provides a platform for rapid iteration and experimentation, allowing developers to test out different prompts and models quickly.
2. Debugging: When developing new LLM applications, LangSmith tracing is enabled by default, providing clear visibility and debugging information at each step of an LLM sequence. This helps identify and root-cause issues when things go wrong.
3. Initial Test Set: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. These test cases can be uploaded in bulk, created on the fly, or exported from application traces.
4. Comparison View: LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application. This helps identify whether or not a variant is performing bet

## Conversation Retrieval Chain

In [24]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [23]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications,

In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)
retrival_chain = create_retrieval_chain(retriever_chain, document_chain)

In [27]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrival_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understan

## Agent

In [30]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [32]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [34]:
tools = [retriever_tool, search]

In [43]:
from langchain_openai import AzureChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = AzureChatOpenAI(azure_deployment="gpt-35-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [44]:
agent_executer.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications or requirements of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different paths and conditions in the code.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the code and specifications to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the code and track the coverage of the test cases. It can provide information on which parts of the code are covered by the tests and which parts are not. This helps in identifying areas that need more testing and improving the overall test coverage.

4. Test Prioritization: LangSmith can priorit

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications or requirements of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different paths and conditions in the code.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the code and specifications to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the code and track the coverage of the test cases. It can provide information on which parts of the code are covered by the tests and which parts are not. This helps in identifying areas that need more testing and improving the overall test coverage.\n\n4. Test Priorit

In [45]:
agent_executer.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health & Activities\nSeasonal Allergies and Pollen Count Forecast\nNo pollen detected in your area\nCold & Flu Forecast\nFlu risk is low in your area\nWe recognize our responsibility to use data and technology for good. recents\nSpecialty Forecasts\n10 Day Weather-San Francisco, CA\nToday\nMon 18 | Day\nConsiderable cloudiness. Tue 19\nTue 19 | Day\nLight rain early...then remaining cloudy with showers in the afternoon. Wed 27\nWed 27 | Day\nOvercast with rain showers at times."}, {'url': 'https://forecast.weath

{'input': 'what is the weather in SF?',
 'output': 'The weather in San Francisco is currently 50°F (10°C) with showers and breezy conditions. You can find more information about the weather in San Francisco on the following websites:\n\n- [Weather.com](https://weather.com/weather/tenday/l/San%20Francisco%20CA%20USCA0987:1:US)\n- [National Weather Service](https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA)\n- [AccuWeather](https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629)\n- [Weather Underground](https://www.wunderground.com/forecast/us/ca/san-francisco)'}

In [46]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executer.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, and overall impact.

2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and effectively presents your qualifications.

3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders and offer suggestions on what information to include in your recommendation letter requests.

4. Application Strategy: LangSmith can help you develop a comprehensive application strategy, including selecting the right law schools, organizing your application materials, and managing deadlines.

5. Interview Preparation: LangSmith can prov

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This includes checking for clarity, coherence, and overall impact.\n\n2. Resume/CV Review: LangSmith can review your resume or CV to ensure it is well-structured, highlights your relevant experiences and achievements, and effectively presents your qualifications.\n\n3. Recommendation Letter Guidance: LangSmith can provide guidance on selecting appropriate recommenders and offer suggestions on what information to include in your recommendation letter requests.\n\n4. Application Strategy: LangSmith can help you develop a comprehensive application strategy, including selecting the righ